In [49]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import google_auth_oauthlib.flow
import googleapiclient.errors
import matplotlib.pyplot as plt
from collections import Counter


import pandas as pd
import os


In [2]:
# Replace 'YOUR_API_KEY' with your actual API key
api_key = 'interApi_key'
youtube = build('youtube', 'v3', developerKey=api_key)

In [6]:
while True:
    playlist_response = youtube.playlists().list(
        part='snippet,contentDetails',
        channelId='inter channel ID here',  # Replace with the channel ID
        maxResults=20,  # Maximum number of items per page
        pageToken=next_page_token  # None for the first request
    ).execute()

    all_playlists.extend(playlist_response['items'])

    # Check if there is a nextPageToken
    next_page_token = playlist_response.get('nextPageToken')
    if not next_page_token:
        break

        

print(f"Total playlists fetched: {len(all_playlists)}")


Total playlists fetched: 41


In [7]:
all_playlists

[{'kind': 'youtube#playlist',
  'etag': 'myzapRcWYo1-60JTYxTIIwSZqN8',
  'id': 'PLBsIgVvbrncDERmAeA0orW6K7YdtQtHdp',
  'snippet': {'publishedAt': '2023-10-11T09:49:22Z',
   'channelId': 'UChu58fbuG4Uepdr6tjrUGTw',
   'title': 'Tea Time',
   'description': '',
   'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/ufODAsrvl1o/default.jpg',
     'width': 120,
     'height': 90},
    'medium': {'url': 'https://i.ytimg.com/vi/ufODAsrvl1o/mqdefault.jpg',
     'width': 320,
     'height': 180},
    'high': {'url': 'https://i.ytimg.com/vi/ufODAsrvl1o/hqdefault.jpg',
     'width': 480,
     'height': 360},
    'standard': {'url': 'https://i.ytimg.com/vi/ufODAsrvl1o/sddefault.jpg',
     'width': 640,
     'height': 480},
    'maxres': {'url': 'https://i.ytimg.com/vi/ufODAsrvl1o/maxresdefault.jpg',
     'width': 1280,
     'height': 720}},
   'channelTitle': 'أحمد بحيري',
   'localized': {'title': 'Tea Time', 'description': ''}},
  'contentDetails': {'itemCount': 1}},
 {'kind': 'youtube#pl

In [8]:
#Fetch Videos for Each Playlist ===== ------ 

def get_playlist_view_count(youtube, playlist_id):
    total_views = 0
    next_page_token = None

    while True:
        # Fetch playlist items
        playlist_items_response = youtube.playlistItems().list(
            part='contentDetails',
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        ).execute()

        # Extract video IDs
        video_ids = [item['contentDetails']['videoId'] for item in playlist_items_response['items']]

        # Fetch video details
        if video_ids:
            videos_response = youtube.videos().list(
                part='statistics',
                id=','.join(video_ids)
            ).execute()

            # Sum up views
            for video in videos_response['items']:
                total_views += int(video['statistics'].get('viewCount', 0))

        next_page_token = playlist_items_response.get('nextPageToken')
        if not next_page_token:
            break

    return total_views


In [9]:
# Assuming all_playlists is your list of playlists
# No need to rewrite the data, just use it directly
df = pd.DataFrame([{
    'playlist_id': playlist['id'],
    'title': playlist['snippet']['title'],
    'published_at': playlist['snippet']['publishedAt'],
    'item_count': playlist['contentDetails']['itemCount'],
    'total_views': get_playlist_view_count(youtube, playlist['id'])
} for playlist in all_playlists])


In [10]:
df

,playlist_id,title,published_at,item_count,total_views
0,PLBsIgVvbrncDERmAeA0orW6K7YdtQtHdp,Tea Time,2023-10-11T09:49:22Z,1,178346
1,PLBsIgVvbrncAu6__cWB8p406QRGG6b5It,Lo-Fi Music,2023-06-29T11:59:07Z,13,393288
2,PLBsIgVvbrncCF_i7QQABvxbx814FddiRX,أحلام عصاري,2023-04-10T11:12:04Z,1,27058
3,PLBsIgVvbrncAHBqnliZ2epLHsUPp3ddwi,الدولة المش مستحيلة,2023-01-18T20:07:13Z,11,1596774
4,PLBsIgVvbrncBhOsJVLc5Nl32GXGHLWytK,غسيل مخ 2.0,2022-12-20T22:08:38Z,13,1382909
5,PLBsIgVvbrncBzaKWMsCdoNkC1OyCF8jBL,بكرة أحـ*ـا,2022-12-16T21:24:46Z,6,1157978
6,PLBsIgVvbrncBkyw6cDkn-jUfmYbKmuFVa,أحدث الحلقات Recent Uploads,2022-08-15T20:09:10Z,969,211272868
7,PLBsIgVvbrncDe3nEMfphXzZIZvTF2oiym,shorts,2022-08-13T10:52:52Z,140,8525741
8,PLBsIgVvbrncBTYDIrNFdv4fwXlYmcnlzL,حدوتة تركية,2022-05-07T16:31:37Z,6,1196056
9,PLBsIgVvbrncCg06bAoXNXPvRcani7U9ES,اسجد واقترب,2022-04-28T21:48:03Z,7,684579


### Get all videos details in the channel after get the playlist


In [1]:
### Get all videos details in the channel after get the playlist



In [12]:
playlist_details = []

for playlist in all_playlists:
    playlist_videos = get_video_details(youtube, playlist['id'])
    for video in playlist_videos:
        video['playlist_id'] = playlist['id']
        video['playlist_title'] = playlist['snippet']['title']
        playlist_details.append(video)

# Create DataFrame from the compiled details
df_all_videos = pd.DataFrame(playlist_details)

df_all_videos


,video_id,video_url,published_at,title,description,views,likes,dislikes,comments,duration,playlist_id,playlist_title
0,ufODAsrvl1o,https://www.youtube.com/watch?v=ufODAsrvl1o,2023-10-11T10:30:09Z,"{Tea Time}(01) Pyramids, Politics, and Paradox...",Good evening everybody.. I know it’s weird hav...,178346,11898,0,1127,PT19M3S,PLBsIgVvbrncDERmAeA0orW6K7YdtQtHdp,Tea Time
1,o6IhbJVv62I,https://www.youtube.com/watch?v=o6IhbJVv62I,2023-06-29T11:59:21Z,{Lo-Fi Music} سيبك انت,"""Seebak Enta""\n\nMusic By: Waleed Saad\n\nCove...",63448,2846,0,252,PT6M17S,PLBsIgVvbrncAu6__cWB8p406QRGG6b5It,Lo-Fi Music
2,pwHcafdfDB8,https://www.youtube.com/watch?v=pwHcafdfDB8,2023-07-07T11:49:55Z,{Lo-Fi Music} Driftwood Serenade,Driftwood Serenade\n\nMusic By: Ahmed Behiry\n...,25922,1345,0,128,PT4M31S,PLBsIgVvbrncAu6__cWB8p406QRGG6b5It,Lo-Fi Music
3,6zRd5qjP1aw,https://www.youtube.com/watch?v=6zRd5qjP1aw,2023-07-13T12:18:34Z,{LoFi Music}(03) و بتسأل يا حبيبي,We Btesaal Ya Habibi\n\nMusic by: Ammar Elshre...,41712,1657,0,106,PT9M38S,PLBsIgVvbrncAu6__cWB8p406QRGG6b5It,Lo-Fi Music
4,KH83xvVDM7g,https://www.youtube.com/watch?v=KH83xvVDM7g,2023-07-18T11:58:28Z,{LoFi Music}(04) ولا عمرك وحشتيني,#shorts\n\nللدعم على Patreon :\nhttps://www.pa...,38264,1768,0,103,PT5M28S,PLBsIgVvbrncAu6__cWB8p406QRGG6b5It,Lo-Fi Music
...,...,...,...,...,...,...,...,...,...,...,...,...
4240,VmecIbJTN8A,https://www.youtube.com/watch?v=VmecIbJTN8A,2012-12-08T17:38:05Z,{غسيل مخ} (٥) الشخصنة,- الشخصنة : هي الهروب من مناقشة الفكرة بالحجة ...,65731,1720,0,195,PT31M10S,PLBsIgVvbrncDyPx2WyqQU6n_8PtIknWYu,(أرشيف)غسيل مخ
4241,HoPVNaUTS5I,https://www.youtube.com/watch?v=HoPVNaUTS5I,2012-11-24T14:40:37Z,{غسيل مخ} (٤) إعمِل حيطة,- مغالطة منطقية تستخدم للهروب من دعم الفكرة بد...,75366,2006,0,102,PT31M56S,PLBsIgVvbrncDyPx2WyqQU6n_8PtIknWYu,(أرشيف)غسيل مخ
4242,9RStzriU8Os,https://www.youtube.com/watch?v=9RStzriU8Os,2012-11-11T20:09:12Z,{غسيل مخ} (٣) بُص العصفورة,"- و نحن نعيش في أزهى عصور ""بص العصفورة"" نحتاج ...",84146,2214,0,197,PT40M28S,PLBsIgVvbrncDyPx2WyqQU6n_8PtIknWYu,(أرشيف)غسيل مخ
4243,Aq2U-ELgnVM,https://www.youtube.com/watch?v=Aq2U-ELgnVM,2012-10-13T08:14:20Z,{غسيل مخ} (١) ثورة الشك..,- القرآن يخاطب العقل ليوقظه\n- الشك الإيجابي.....,122401,4015,0,137,PT11M58S,PLBsIgVvbrncDyPx2WyqQU6n_8PtIknWYu,(أرشيف)غسيل مخ


In [13]:
df_all_videos.sample(50)

,video_id,video_url,published_at,title,description,views,likes,dislikes,comments,duration,playlist_id,playlist_title
831,JB30Yj5HyN8,https://www.youtube.com/watch?v=JB30Yj5HyN8,2022-08-09T22:25:43Z,{تعاشب شاي}(1050) يا احنا يا انتو في البلد دي!,لو حضرتك استأذنت المدام تعمل لك كباية شاي معتب...,200153,7646,0,90,PT12M51S,PLBsIgVvbrncBkyw6cDkn-jUfmYbKmuFVa,أحدث الحلقات Recent Uploads
4215,U6bx9mW3qkk,https://www.youtube.com/watch?v=U6bx9mW3qkk,2013-02-10T06:18:08Z,{حكاية العمر كله} (٤) إما يبلغن عندك الكبر,الأب الشاب.. الواعي.. اللي عارف اللي ليه و الل...,16965,552,0,41,PT24M2S,PLBsIgVvbrncDsN4RZtXISW9fnmOw9Arna,حكاية العمر كله
1450,QjC8xjAcwjs,https://www.youtube.com/watch?v=QjC8xjAcwjs,2019-11-01T17:00:09Z,{جسم كمنجه}(01) اوعى يغرك جسمك!,للدعم على Patreon :\nhttps://www.patreon.com/a...,395979,18977,0,1279,PT18M26S,PLBsIgVvbrncDUt3RTUxlRibaJGaTCCTdS,جسم كمنجه
941,PYMcr1WTCJI,https://www.youtube.com/watch?v=PYMcr1WTCJI,2022-05-27T20:48:08Z,{اسجد واقترب}(06) فاتحة العيون و القلوب,إذا كانت الصلاة هي عماد الدين.. فـ”الفاتحة”.. ...,53740,2600,0,116,PT20M13S,PLBsIgVvbrncBkyw6cDkn-jUfmYbKmuFVa,أحدث الحلقات Recent Uploads
416,ItsAAiVbS6E,https://www.youtube.com/watch?v=ItsAAiVbS6E,2023-04-12T17:22:00Z,{تعاشب شاي}(1368) هل تسريبات البنتاجون حقيقية؟,دليل سريع لما تقوله تقارير المخابرات الأمريكية...,147442,6776,0,114,PT8M9S,PLBsIgVvbrncBkyw6cDkn-jUfmYbKmuFVa,أحدث الحلقات Recent Uploads
1229,OBLcGAhk7RI,https://www.youtube.com/watch?v=OBLcGAhk7RI,2021-03-15T18:37:56Z,{حدوتة عربية}(06) الدولة الأموية.. حلوة بس شقية,قرب نهاية سنة 691 ميلادية.. سافر عبد الملك ، ...,354442,11603,0,553,PT23M18S,PLBsIgVvbrncBI-GBODIFomvD6RVkJ618b,حدوتة عربية
2352,X11l6-C2Poc,https://www.youtube.com/watch?v=X11l6-C2Poc,2016-05-10T06:07:27Z,{الأسبوع ف كيس} (44) ماتخافشي,الصفحة الرسمية لاحمد بحيري علي الفيسبوك \nhttp...,462793,11313,0,682,PT27M50S,PLBsIgVvbrncAeJMvwCpzZg-ow581Oh13M,الأسبوع ف كيس
4207,7WM3yn9RBV0,https://www.youtube.com/watch?v=7WM3yn9RBV0,2014-07-01T10:19:26Z,{تعاشَّب شاي} (06) حنفي في #التراويح,كتب هذه الحلقة : د. حنان لاشين (أم البنين)\nht...,37327,1421,0,29,PT3M23S,PLBsIgVvbrncB0M8dlzAbvjl79Nu8v_BW4,تعاشَّب شاي
522,R6c-Mf2z5jg,https://www.youtube.com/watch?v=R6c-Mf2z5jg,2023-02-09T18:32:36Z,{ساس يسوس}(104) قالوا للأثيوبي احلف!,زي ما اتكلمنا الحلقة اللي فاتت عن تركيا وقولنا...,221869,6371,0,256,PT24M54S,PLBsIgVvbrncBkyw6cDkn-jUfmYbKmuFVa,أحدث الحلقات Recent Uploads
1853,WRGj5HxevDQ,https://www.youtube.com/watch?v=WRGj5HxevDQ,2021-01-17T20:42:11Z,{عرفوه بالعقل}(43) هل الإسلام دين عنف؟,أحداث العنف اللي بتتّم على أيد الجماعات المتطر...,259970,10994,0,509,PT32M35S,PLBsIgVvbrncCUHtc-Gd9d0t5wBXwC0_DO,عرفوه بالعقل!


In [14]:
# convert Youtube duration to seconds
#pip install isodate

import isodate

In [15]:
df_all_videos ['durationSecs'] = df_all_videos ['duration'].apply (lambda x: isodate.parse_duration (x))
df_all_videos ['durationSecs'] = df_all_videos ['durationSecs'].astype('timedelta64[s]')

In [16]:
# Remove "0 days" form the durationSecs column ---- 

df_all_videos['durationSecs'] = df_all_videos['durationSecs'].dt.total_seconds()


In [17]:
df_all_videos.sample(50)

,video_id,video_url,published_at,title,description,views,likes,dislikes,comments,duration,playlist_id,playlist_title,durationSecs
1571,U4CrE2CNMwQ,https://www.youtube.com/watch?v=U4CrE2CNMwQ,2020-07-15T20:43:36Z,{الحراق}(23) راح النور,لما بتبقى قاعد في بيتك بالليل منور نور الشقة ك...,203824,9940,0,437,PT16M13S,PLBsIgVvbrncBJbmFFzhjJfzK9aLoMuouz,الحرَّاق,973.0
2816,izCBUIPSLfU,https://www.youtube.com/watch?v=izCBUIPSLfU,2023-04-10T19:07:24Z,{تعاشب شاي}(1366) تعالى اسيحك!,طول عمرنا بنسأل ليه السياحة عندنا بعافية .. بل...,137980,5809,0,116,PT13M9S,PLBsIgVvbrncB0M8dlzAbvjl79Nu8v_BW4,تعاشَّب شاي,789.0
2460,PdYA7UsEfrk,https://www.youtube.com/watch?v=PdYA7UsEfrk,2020-10-21T21:09:06Z,{ساس يسوس}(74) لينين ملاح و معدينا!,اتكلمنا في الحلقات اللي فاتت عن تعريف المؤسسات...,194670,7035,0,283,PT27M4S,PLBsIgVvbrncC_hFoIcgf8_NyqLkF1nkba,ساسا ياسوسو,1624.0
698,SYyqgGxKCJI,https://www.youtube.com/watch?v=SYyqgGxKCJI,2022-10-21T20:22:40Z,{تعاشب شاي}(1149) مانجيلكوش في أوسكار!,في أواخر سبتمبر اللي فات ، وفي الوقت اللي بيست...,187433,7494,0,119,PT16M51S,PLBsIgVvbrncBkyw6cDkn-jUfmYbKmuFVa,أحدث الحلقات Recent Uploads,1011.0
3819,3OAtCjbRxOU,https://www.youtube.com/watch?v=3OAtCjbRxOU,2020-05-15T18:42:07Z,{تعاشب شاي}(383) جردل الوطنية,انت كمشاهد مصري تبقي لسه يا عيني من كام شهر شا...,548879,23373,0,740,PT16M39S,PLBsIgVvbrncB0M8dlzAbvjl79Nu8v_BW4,تعاشَّب شاي,999.0
3499,64UkmKWYFB4,https://www.youtube.com/watch?v=64UkmKWYFB4,2021-07-17T18:54:04Z,{تعاشب شاي}(686) بتحبسوا الصاروخ ليه؟,دايما بتعدي عليك اعلانات الشغل .. وتلاقيها كله...,248332,10666,0,362,PT14M41S,PLBsIgVvbrncB0M8dlzAbvjl79Nu8v_BW4,تعاشَّب شاي,881.0
1142,q0o3JU8__kQ,https://www.youtube.com/watch?v=q0o3JU8__kQ,2022-11-05T13:21:28Z,وزيرة الهجرة: الدول الغربية بتعاني وربنا يكون ...,وزيرة الهجرة: الدول الغربية بتعاني وربنا يكون ...,17739,1323,0,13,PT51S,PLBsIgVvbrncDe3nEMfphXzZIZvTF2oiym,shorts,51.0
422,fM79pGduElA,https://www.youtube.com/watch?v=fM79pGduElA,2023-04-09T17:26:11Z,{تعاشب شاي}(1346) محاكمة القرن,الجدع اللي اسمه ترمب ده من يوم ما طلع لنا في ا...,155345,6358,0,100,PT12M18S,PLBsIgVvbrncBkyw6cDkn-jUfmYbKmuFVa,أحدث الحلقات Recent Uploads,738.0
3642,qdpaPz60Y4w,https://www.youtube.com/watch?v=qdpaPz60Y4w,2021-02-16T18:27:11Z,{تعاشب شاي}(543) لولب من بلح!,سنة 2019 صدر تقرير من وزارة التضامن الاجتماعي ...,307786,12162,0,404,PT21M10S,PLBsIgVvbrncB0M8dlzAbvjl79Nu8v_BW4,تعاشَّب شاي,1270.0
2935,3Z58Rght_Qk,https://www.youtube.com/watch?v=3Z58Rght_Qk,2023-01-14T21:50:14Z,{تعاشب شاي}(1248) وداعاً حديقة الحيوان..,جنينة الحيوانات المصرية التاريخية الي بتعتبر م...,221847,8778,0,278,PT13M56S,PLBsIgVvbrncB0M8dlzAbvjl79Nu8v_BW4,تعاشَّب شاي,836.0


In [18]:
# Fix the Date and time in published_at column  ------
# Split the datetime string into separate date and time columns


# Remove the 'Z' at the end of the datetime string
df_all_videos['published_at'] = df_all_videos['published_at'].str.rstrip('Z')

# Split the datetime string into separate date and time columns
df_all_videos[['date', 'time']] = df_all_videos['published_at'].str.split('T', expand=True)


df_all_videos.sample(50)

,video_id,video_url,published_at,title,description,views,likes,dislikes,comments,duration,playlist_id,playlist_title,durationSecs,date,time
2007,-dXJwBFLuPk,https://www.youtube.com/watch?v=-dXJwBFLuPk,2023-07-06T20:48:08,{الأسبوع ف كيس}(351) عيد ميلاد بلحي أنا,مش الأسبوع ده كان عيد ميلاد الانقلاب .. اه وال...,387528,10585,0,331,PT39M15S,PLBsIgVvbrncAeJMvwCpzZg-ow581Oh13M,الأسبوع ف كيس,2355.0,2023-07-06,20:48:08
3076,JEEC9nMgF00,https://www.youtube.com/watch?v=JEEC9nMgF00,2022-09-22T19:12:56,{تعاشب شاي}(1107) روبي.. و التهمة شريف,في الوقت اللي كان الأمير تميم بيستضيف عزيز مصر...,168821,7055,0,111,PT13M30S,PLBsIgVvbrncB0M8dlzAbvjl79Nu8v_BW4,تعاشَّب شاي,810.0,2022-09-22,19:12:56
652,yNldGplr3Qg,https://www.youtube.com/watch?v=yNldGplr3Qg,2022-11-18T21:29:16,{يلا مانفهمش ف الكورة}(06) يلا مانزعلش على كاس...,من اربع سنين حصلت معجزة شغلت العالم أجمع و خلت...,206657,7737,0,193,PT9M34S,PLBsIgVvbrncBkyw6cDkn-jUfmYbKmuFVa,أحدث الحلقات Recent Uploads,574.0,2022-11-18,21:29:16
2550,GuzandY3VLA,https://www.youtube.com/watch?v=GuzandY3VLA,2023-12-11T21:30:08,{تعاشب شاي}(1632) فلسطين.. بين مخلوع و مبلوح,سنين طويلة من العداوة التاريخية كانت بين مصر و...,226783,8857,0,185,PT21M14S,PLBsIgVvbrncB0M8dlzAbvjl79Nu8v_BW4,تعاشَّب شاي,1274.0,2023-12-11,21:30:08
685,exPtBaOfPVU,https://www.youtube.com/watch?v=exPtBaOfPVU,2022-10-29T18:43:29,{تعاشب شاي}(1160) لماذا يشاهد السيسي الحسيني ب...,في وسط اسهال الكلام اللي صايب السيسي اليومين د...,409681,12974,0,330,PT27M40S,PLBsIgVvbrncBkyw6cDkn-jUfmYbKmuFVa,أحدث الحلقات Recent Uploads,1660.0,2022-10-29,18:43:29
2119,R6RuDIZhGqM,https://www.youtube.com/watch?v=R6RuDIZhGqM,2020-12-24T19:13:27,{الأسبوع ف كيس}(247) التكريم باليد.. ولا الحوج...,للدعم على Patreon :\nhttps://www.patreon.com/a...,449141,17184,0,422,PT27M25S,PLBsIgVvbrncAeJMvwCpzZg-ow581Oh13M,الأسبوع ف كيس,1645.0,2020-12-24,19:13:27
2479,U6ue442-PH4,https://www.youtube.com/watch?v=U6ue442-PH4,2020-05-08T19:32:20,{ساسا ياسوسو}(58) إنقلاب لا يترنح,واتكلمنا فى الحلقة اللي فاتت عن نجم الانقلاب ا...,510952,14307,0,834,PT42M32S,PLBsIgVvbrncC_hFoIcgf8_NyqLkF1nkba,ساسا ياسوسو,2552.0,2020-05-08,19:32:20
794,vef4CN3mNyg,https://www.youtube.com/watch?v=vef4CN3mNyg,2022-08-21T21:17:47,{تعاشب شاي}(1065) قالوا الدولار قادر و له ألف ...,طبعا كلنا ملهيين و مشغولين في متابعة اخبار الد...,206198,7679,0,122,PT12M49S,PLBsIgVvbrncBkyw6cDkn-jUfmYbKmuFVa,أحدث الحلقات Recent Uploads,769.0,2022-08-21,21:17:47
1151,-hLfvcYAG0c,https://www.youtube.com/watch?v=-hLfvcYAG0c,2022-11-08T15:52:45,الفنان احمد العوضي يدير ندوة في جامعة حلوان عن...,الفنان احمد العوضي يدير ندوة في جامعة حلوان عن...,26509,1282,0,29,PT1M1S,PLBsIgVvbrncDe3nEMfphXzZIZvTF2oiym,shorts,61.0,2022-11-08,15:52:45
2040,AIPvGKt849A,https://www.youtube.com/watch?v=AIPvGKt849A,2022-10-07T21:43:20,{الأسبوع ف كيس}(320) زبابيط الله.. أحباب الله,كل سنة وانتوا طيبين..العام الدراسي الجديد بدأ ...,417461,12851,0,388,PT40M31S,PLBsIgVvbrncAeJMvwCpzZg-ow581Oh13M,الأسبوع ف كيس,2431.0,2022-10-07,21:43:20


In [19]:
# Convert 'date' column to datetime
df_all_videos['date'] = pd.to_datetime(df_all_videos['date'])

# Convert 'time' column to datetime (set the format if needed)
df_all_videos['time'] = pd.to_datetime(df_all_videos['time'], format='%H:%M:%S')


In [20]:
#df_all_videos.sort_values(by='published_at', ascending=False)

In [21]:
df_all_videos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4245 entries, 0 to 4244
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   video_id        4245 non-null   object        
 1   video_url       4245 non-null   object        
 2   published_at    4245 non-null   object        
 3   title           4245 non-null   object        
 4   description     4245 non-null   object        
 5   views           4245 non-null   int64         
 6   likes           4245 non-null   int64         
 7   dislikes        4245 non-null   int64         
 8   comments        4245 non-null   int64         
 9   duration        4245 non-null   object        
 10  playlist_id     4245 non-null   object        
 11  playlist_title  4245 non-null   object        
 12  durationSecs    4245 non-null   float64       
 13  date            4245 non-null   datetime64[ns]
 14  time            4245 non-null   datetime64[ns]
dtypes: d

In [22]:
# check the nall values --------

df_all_videos.isnull().sum()

video_id          0
video_url         0
published_at      0
title             0
description       0
views             0
likes             0
dislikes          0
comments          0
duration          0
playlist_id       0
playlist_title    0
durationSecs      0
date              0
time              0
dtype: int64

In [23]:
df_all_videos.describe().round(2)

,views,likes,dislikes,comments,durationSecs,date,time
count,4245.00,4245.00,4245.0,4245.00,4245.00,4245,4245
mean,244503.31,9382.45,0.0,330.40,994.91,2021-06-16 06:41:58.727915264,1900-01-01 17:29:24.335217920
min,6320.00,167.00,0.0,0.00,13.00,2012-04-22 00:00:00,1900-01-01 00:06:46
25%,141805.00,6167.00,0.0,146.00,701.00,2020-04-28 00:00:00,1900-01-01 16:32:50
50%,202567.00,8218.00,0.0,239.00,837.00,2022-03-12 00:00:00,1900-01-01 19:11:10
75%,295023.00,11462.00,0.0,407.00,1147.00,2023-01-09 00:00:00,1900-01-01 20:31:58
max,3498058.00,87846.00,0.0,8773.00,5736.00,2023-12-19 00:00:00,1900-01-01 23:53:05
std,197997.14,5932.42,0.0,363.15,587.32,NaN,NaN


In [24]:
df_all_videos.nunique()

video_id          3071
video_url         3071
published_at      3062
title             3039
description       2553
views             3172
likes             2778
dislikes             1
comments           892
duration          1365
playlist_id         41
playlist_title      41
durationSecs      1365
date              1850
time              2861
dtype: int64

#### Define a Function to Fetch Comments for all Videos in the channel

In [25]:
def get_comments(youtube, video_id):
    comments = []
    next_page_token = None

    while True:
        # Fetch comments from the video
        comments_response = youtube.commentThreads().list(
            part='snippet',
            videoId=video_id,
            maxResults=100,  # API allows max 100
            pageToken=next_page_token,
            textFormat='plainText'
        ).execute()

        # Extract comment text and other details if needed
        for item in comments_response['items']:
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            comments.append(comment)

        next_page_token = comments_response.get('nextPageToken')
        if not next_page_token:
            break

    return comments







### Fetch all comments for a plylist for exsapmles " بلاي ليست عرفوه بالعقل"

In [27]:
playlist_id = "Playlist id here" # Playlist id ---  عرفوه بالعقل 

df_playlist_videos = df_all_videos[df_all_videos['playlist_id'] == playlist_id]


In [28]:
# Fetch Comments and Complete Data ---

all_comments_data = []

#for _, video in df_playlist_videos.iterrows():  <--- this code alternative for the next line both working
    
for index, video in df_playlist_videos.iterrows():

    video_id = video['video_id']
    video_comments = get_comments(youtube, video_id)
    
    for comment in video_comments:
        comment_data = {
            'video_id': video_id,
            'video_url': video['video_url'],
            'video_title': video['title'],
            'views': video['views'],
            'likes': video['likes'],
            'date':video['date'],
            'time':video['time'],
            'durationSecs': video['durationSecs'],
            'comment': comment,
             #'published_at': video['published_at'], I seprat it in two columns data and time
             #  'dislikes': video['dislikes'], based on the yourube apis doesn't provied the dislike --
        }
        all_comments_data.append(comment_data)
        


In [29]:
# Create a DataFrame for Comments Data --- 

df_comments = pd.DataFrame(all_comments_data)

df_comments


,video_id,video_url,video_title,views,likes,date,time,durationSecs,comment
0,gALjG2pMNWQ,https://www.youtube.com/watch?v=gALjG2pMNWQ,{عرفوه بالعقل}(47) هل يجوز الجواز إذا جاز؟,170752,7718,2021-12-27,1900-01-01 19:19:57,1133.0,اشترك في القناة الرسمية على تليجرام:\nhttps://...
1,gALjG2pMNWQ,https://www.youtube.com/watch?v=gALjG2pMNWQ,{عرفوه بالعقل}(47) هل يجوز الجواز إذا جاز؟,170752,7718,2021-12-27,1900-01-01 19:19:57,1133.0,ما أجمل اسلوبك وطريقة طرحك للمواضيع استاذ احمد
2,gALjG2pMNWQ,https://www.youtube.com/watch?v=gALjG2pMNWQ,{عرفوه بالعقل}(47) هل يجوز الجواز إذا جاز؟,170752,7718,2021-12-27,1900-01-01 19:19:57,1133.0,ربنا يباركلك .. استحالة تكون بتلعب بالالفاظ لا...
3,gALjG2pMNWQ,https://www.youtube.com/watch?v=gALjG2pMNWQ,{عرفوه بالعقل}(47) هل يجوز الجواز إذا جاز؟,170752,7718,2021-12-27,1900-01-01 19:19:57,1133.0,جميل جدا.. بوركت جهودك استاذ
4,gALjG2pMNWQ,https://www.youtube.com/watch?v=gALjG2pMNWQ,{عرفوه بالعقل}(47) هل يجوز الجواز إذا جاز؟,170752,7718,2021-12-27,1900-01-01 19:19:57,1133.0,🌹🌹👍
...,...,...,...,...,...,...,...,...,...
1146,wMwLcSh5aRw,https://www.youtube.com/watch?v=wMwLcSh5aRw,{عرفوه بالعقل}(53) نسمع أغاني ولا لأ؟,110422,5408,2022-03-14,1900-01-01 20:00:20,1453.0,الاول
1147,wMwLcSh5aRw,https://www.youtube.com/watch?v=wMwLcSh5aRw,{عرفوه بالعقل}(53) نسمع أغاني ولا لأ؟,110422,5408,2022-03-14,1900-01-01 20:00:20,1453.0,جميل جدا
1148,wMwLcSh5aRw,https://www.youtube.com/watch?v=wMwLcSh5aRw,{عرفوه بالعقل}(53) نسمع أغاني ولا لأ؟,110422,5408,2022-03-14,1900-01-01 20:00:20,1453.0,لأ
1149,wMwLcSh5aRw,https://www.youtube.com/watch?v=wMwLcSh5aRw,{عرفوه بالعقل}(53) نسمع أغاني ولا لأ؟,110422,5408,2022-03-14,1900-01-01 20:00:20,1453.0,بحب برضو


In [30]:
# Convert 'time' column to datetime (set the format if needed)
#df_comments['time'] = pd.to_datetime(df_comments['time']).dt.time

df_comments['time'] = pd.to_datetime(df_comments['time'], format='%H:%M:%S').dt.time



In [36]:
df_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1151 entries, 0 to 1150
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   video_id      1151 non-null   object        
 1   video_url     1151 non-null   object        
 2   video_title   1151 non-null   object        
 3   views         1151 non-null   int64         
 4   likes         1151 non-null   int64         
 5   date          1151 non-null   datetime64[ns]
 6   time          1151 non-null   object        
 7   durationSecs  1151 non-null   float64       
 8   comment       1151 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(5)
memory usage: 81.1+ KB


In [32]:
df_comments.sample(50)

,video_id,video_url,video_title,views,likes,date,time,durationSecs,comment
462,-j-Go-9Syjo,https://www.youtube.com/watch?v=-j-Go-9Syjo,{عرفوه بالعقل}(49) هل حرم القرآن الموسيقى؟,169796,7665,2022-01-16,20:36:47,1464.0,الله يفتح عليك
693,in1b0OdSRnE,https://www.youtube.com/watch?v=in1b0OdSRnE,{عرفوه بالعقل}(50) هل حرم النبي ﷺ الموسيقى؟,152405,6778,2022-01-23,20:35:50,1796.0,اللهم اعنا على ذكرك وشكرك وحسن عبادتك
1068,wMwLcSh5aRw,https://www.youtube.com/watch?v=wMwLcSh5aRw,{عرفوه بالعقل}(53) نسمع أغاني ولا لأ؟,110422,5408,2022-03-14,20:00:20,1453.0,متحمسة جدا اعرف الموضوع الجديد اللى هتتناوله ...
131,gALjG2pMNWQ,https://www.youtube.com/watch?v=gALjG2pMNWQ,{عرفوه بالعقل}(47) هل يجوز الجواز إذا جاز؟,170752,7718,2021-12-27,19:19:57,1133.0,وانت بالف خير يارب
248,DK2P1wK7F-0,https://www.youtube.com/watch?v=DK2P1wK7F-0,{عرفوه بالعقل}(48) تيجي نوزن برة؟,126291,5933,2022-01-08,12:56:17,939.0,اصحابي كالنجوم
356,-j-Go-9Syjo,https://www.youtube.com/watch?v=-j-Go-9Syjo,{عرفوه بالعقل}(49) هل حرم القرآن الموسيقى؟,169796,7665,2022-01-16,20:36:47,1464.0,محاضرة قيمة احسنت يا استاذ احمد لبحيري
352,-j-Go-9Syjo,https://www.youtube.com/watch?v=-j-Go-9Syjo,{عرفوه بالعقل}(49) هل حرم القرآن الموسيقى؟,169796,7665,2022-01-16,20:36:47,1464.0,أحسنت
673,in1b0OdSRnE,https://www.youtube.com/watch?v=in1b0OdSRnE,{عرفوه بالعقل}(50) هل حرم النبي ﷺ الموسيقى؟,152405,6778,2022-01-23,20:35:50,1796.0,ما فهمته اني لا حرج ان اسمع الغناء بشرط الا ي...
92,gALjG2pMNWQ,https://www.youtube.com/watch?v=gALjG2pMNWQ,{عرفوه بالعقل}(47) هل يجوز الجواز إذا جاز؟,170752,7718,2021-12-27,19:19:57,1133.0,ياعزيزي اغلبنا مقييدين
595,in1b0OdSRnE,https://www.youtube.com/watch?v=in1b0OdSRnE,{عرفوه بالعقل}(50) هل حرم النبي ﷺ الموسيقى؟,152405,6778,2022-01-23,20:35:50,1796.0,أرجو من حضرتك بحث في موضوع البنوك .. وشكرا لمج...


In [ ]:
# Convert DataFrame to CSV -----
df_comments.to_csv('df_comments_arafo_belakel.csv', index=False)


In [ ]:
# Scope for YouTube Analytics API

scopes = ['https://www.googleapis.com/auth/yt-analytics.readonly']

def main():
    api_service_name = "youtubeAnalytics"
    api_version = "v2"
    client_secrets_file = "client_secret_2.json"
    
    # Get credentials and create an API client
    flow = InstalledAppFlow.from_client_secrets_file(client_secrets_file, scopes=scopes)

    #flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
    credentials = flow.run_local_server()
    youtube_analytics = googleapiclient.discovery.build(
        api_service_name, api_version, credentials=credentials)

    request = youtube_analytics.reports().query(
        dimensions="country",
        startDate="2023-11-01",
        endDate="2023-11-30",
        ids="channel ID here",
        metrics="views,estimatedMinutesWatched,averageViewDuration,averageViewPercentage,subscribersGained",
        sort="-estimatedMinutesWatched"
        
    )
    response = request.execute()

    print(response)

if __name__ == "__main__":
    main()
    